# Building a Goodreads recommendation system
**Capstone Project 2 Final Report**

## Background

Goodreads is a social media platform that allows users to rate and review books as well as see what their friends are reading, rating, and reviewing. Users can store books on ‘shelves’ based on what they’ve read or what they want to read. 

Currently, the Goodreads recommendation system uses a user’s shelves to suggest books they might be interested in reading next. However, as I've learned from personal experience on the platform, the system does not always offer helpful suggestions since most of the books it recommends are obscure and do not appear to be based on what a user previously rated or what their friends have read.

Is it possible to create a more useful recommendation system for readers? 

## The Data 

To answer this question I will be using the Goodbooks __[datasets](https://github.com/zygmuntz/goodbooks-10k)__ provided by Github user __[zygmuntz](https://github.com/zygmuntz)__. 

The Goodbooks dataset includes over six million ratings of ten thousand books on Goodreads. It is separated into three different files 

* **Ratings:** Contains nearly 6 million user ratings from 53,424 users 
* **To-Read:** Contains nearly 1 million books that users added to their 'to-read' shelf 
* **Books:** Contains all of the meta data for 10,000 books. The metadata includes: title, author, number of ratings, number of each type of rating, and more 

For the purposes of this project, I will be using the Ratings and Books datasets.

### Loading the Data


In [1]:
# Import libraries
# ----------------

# Pandas
import pandas as pd

# Numpy
import numpy as np

# SKLearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
# Load Datasets
# ----------------

# Ratings
ratings_data = "../data/raw/ratings.csv"
ratings = pd.read_csv(ratings_data)

# Books
books_data = "../data/raw/books.csv"
books = pd.read_csv(books_data)
books = books[['book_id', 'title']] ## I only need the book_id and title columns for modeling 

# Merge ratings and books datasets
merged = pd.merge(books, ratings)
print("Number of books in dataset: ",len(merged.book_id.value_counts()))
print("Number of users in dataset: ",len(merged.user_id.value_counts()))
merged.head()

Number of books in dataset:  10000
Number of users in dataset:  53424


,book_id,title,user_id,rating
0,1,"The Hunger Games (The Hunger Games, #1)",2886,5
1,1,"The Hunger Games (The Hunger Games, #1)",6158,5
2,1,"The Hunger Games (The Hunger Games, #1)",3991,4
3,1,"The Hunger Games (The Hunger Games, #1)",5281,5
4,1,"The Hunger Games (The Hunger Games, #1)",5721,5


## Building a simple model

A simple model recommends the same books to all users regardless of their reading or rating history. The recommendations are usually based on the most popular or highest rated items or a score that combines the two. For my simple model, I will be giving each book a weighted score. This will prevent a book with only 10 reviews that are all 5 stars from skewing my results.

To build my model I will be adapting DataCamp's _[IMDB 250 tutorial](https://www.datacamp.com/community/tutorials/recommender-systems-python)_ which gives each item a weighted score using the following formula: 

<img src="https://github.com/ameenamarie/Goodreads-Recommendation-System/blob/master/reports/figures/image1.png?raw=true}" title="Weighted Score" width=400 />

* _v_ number of ratings a book has received;
* _m_ the minimum number of ratings a book needs to receive to be included in the model;
* _R_ the average rating for the book; And
* _C_ the average rating for all books

To start I created a column with the average number of ratings that a book received (*v*) and the average rating that each book received (*R*). I then calculated the average rating for all books (*C*) and identified the books that fall in the 90th quantile (*m*). 

In [4]:
# Create a column for the number of ratings a book has received called 'ratings_count'
merged['ratings_count'] = merged['book_id'].groupby(merged['book_id']).transform('count')

# Create a column for the average rating a book has received called 'average_rating'
merged['average_rating'] = merged['rating'].groupby(merged['book_id']).transform('mean')

# Calculate the average rating for all books
C = merged['rating'].mean()

# Calculate the minimum number of ratings a book needs to receive in order to be included in the model
m = merged['ratings_count'].quantile(0.90)

# Filter the dataset based on value m
filtered = merged.copy().loc[merged['ratings_count'] >= m]

I then built a function that gives a weighted score to each book using the formula above:

In [5]:
# Create a function that gives a weighted score to each book
def weighted_score(x, m=m, C=C):
    v = x['ratings_count']
    R = x['average_rating']
    # Calculation based on an IMDB formula 
    return (v/(v+m) * R) + (m/(m+v) * C)

# Create a 'score' column and give each book a weighted score
filtered['score'] = filtered.apply(weighted_score, axis=1)
filtered.head()

,book_id,title,user_id,rating,ratings_count,average_rating,score
0,1,"The Hunger Games (The Hunger Games, #1)",2886,5,22806,4.279707,4.170325
1,1,"The Hunger Games (The Hunger Games, #1)",6158,5,22806,4.279707,4.170325
2,1,"The Hunger Games (The Hunger Games, #1)",3991,4,22806,4.279707,4.170325
3,1,"The Hunger Games (The Hunger Games, #1)",5281,5,22806,4.279707,4.170325
4,1,"The Hunger Games (The Hunger Games, #1)",5721,5,22806,4.279707,4.170325


Once every book had a score, I dropped the duplicates and ranked the books by score. I then identified the ten books with the higest score. 

In [8]:
# Sort books based on score
top10 = filtered.sort_values('score', ascending=False)

# Drop duplicate books
top10 = top10.drop_duplicates(subset='book_id', keep="first")

#Print the top 10 books
top10[['title', 'ratings_count', 'average_rating', 'score']].head(10)

,title,ratings_count,average_rating,score
379644,Harry Potter and the Deathly Hallows (Harry Po...,15304,4.525941,4.287004
402878,Harry Potter and the Half-Blood Prince (Harry ...,15081,4.443339,4.235129
354945,Harry Potter and the Goblet of Fire (Harry Pot...,15523,4.430780,4.231090
262333,Harry Potter and the Prisoner of Azkaban (Harr...,15855,4.418732,4.226258
25217,Harry Potter and the Sorcerer's Stone (Harry P...,21850,4.351350,4.216248
63196,To Kill a Mockingbird,19088,4.329369,4.188958
310931,Harry Potter and the Order of the Phoenix (Har...,15258,4.358697,4.185378
451504,The Help,12727,4.382887,4.179612
20785,"The Hunger Games (The Hunger Games, #1)",22806,4.279707,4.170325
535323,"A Game of Thrones (A Song of Ice and Fire, #1)",10692,4.339880,4.137317


Wow! It looks like everyone really loves Harry Potter. 

If I were to use this model as my recommendation system, Goodreads would recommend these 10 books to every user regardless of their rating or reading history. However, I do not want to recommend Harry Potter to every single user, so I decided to build a collaborative model to build more specific recommendations. 

I can use this simple recommendation model to recommend books to "cold start" users. That is, users who have just joined Goodreads and do not yet have a reading or rating history on the platform. 

## Building a collaborative model from scratch

Collaborative models make recommendations by comparing users and books to each other and grouping by similarity. This is accomplished by measuring the distance between each user and each item in order to group similar users and items together. The model makes multiple calculations for every item in the dataset, so it is best to have a smaller dataset. 

I started by filtering the data to only include users that have reviewed at least 175 books.

In [9]:
# Filter data to only include users who have reviewed at least 175 books
collab = merged
collab['number_of_reviews'] = collab['user_id'].groupby(collab['user_id']).transform('count') 
collab = collab[collab['number_of_reviews'] >= 175]
collab = collab.drop('number_of_reviews', 1)

When I removed users who have rated fewer than 175 books, I had to create a new user index and a new book index so that I could place the ratings for each user and book into a matrix. 

In [10]:
# Create a new column for user_index and book_index
collab = collab.assign(user_index=(collab['user_id']).astype('category').cat.codes)
collab = collab.assign(book_index=(collab['book_id']).astype('category').cat.codes)

n_users = collab.user_id.unique().shape[0]
n_items = collab.book_id.unique().shape[0]
print('DataFrame shape: {}'.format(collab.shape))
print('Number of unique users:', n_users)
print('Number of books:', n_items)
collab.head()

DataFrame shape: (108497, 8)
Number of unique users: 596
Number of books: 7574


,book_id,title,user_id,rating,ratings_count,average_rating,user_index,book_index
200,1,"The Hunger Games (The Hunger Games, #1)",6323,4,22806,4.279707,60,0
459,1,"The Hunger Games (The Hunger Games, #1)",10140,4,22806,4.279707,120,0
461,1,"The Hunger Games (The Hunger Games, #1)",10146,5,22806,4.279707,121,0
616,1,"The Hunger Games (The Hunger Games, #1)",10792,5,22806,4.279707,134,0
631,1,"The Hunger Games (The Hunger Games, #1)",10120,3,22806,4.279707,119,0


The collaborative model requires a matrix using the user_index, book_index, and rating in order to measure the distance between each item and user. I created a training and test matrix. 

In [11]:
# Rearrange the order of the columns to prep for modeling
collab = collab[['user_index', 'book_index', 'rating', 'book_id', 'title', 'user_id', 'ratings_count',
                  'average_rating']]

#from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(collab, test_size=0.25)

#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

Once everything was placed in a train and test matrix, I was able to calculate the similarity between each user and item using pairwise_distance. 

In [12]:
# Calculate the similarity using pairwise_distance
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

Now I'm ready to make predictions!

In [13]:
# Define a function to make predictions
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [14]:
# Make Predictions
user_prediction = predict(train_data_matrix, user_similarity, type='user')
item_prediction = predict(train_data_matrix, item_similarity, type='item')

### Evaluate the model 

Let's see how the model performed:

In [15]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [16]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.60414917018056
Item-based CF RMSE: 3.90816668003082


Not bad!

## Singular Vector Decomposition Model

The Singular Vector Decomposition model breaks a matrix (like the one above) into its component parts. 

<img src="https://github.com/ameenamarie/Goodreads-Recommendation-System/blob/master/reports/figures/image2.png?raw=true}" title="SVD Formula", width=150 />

* *R* is the user ratings matrix;
* *U* is the user "features" matrix;
* *Σ* is the diagonal matrix of singular values (weights); And
* *Vt* is the books features matrix 

First I filtered the dataset to only include users who have rated at least 175 books like I did with the collaborative model. I then converted the dataframe to a pivot table with one row for every user and one column for every book. The value is the rating that the user gave a particular book. 

In [17]:
svd = merged

# Filter data to only include users who have reviewed at least 175 books 
svd['number_of_reviews'] = svd['user_id'].groupby(svd['user_id']).transform('count') 
svd = svd[svd['number_of_reviews'] >= 175]
svd = svd.drop('number_of_reviews', 1)

# Create a pivot table of users and ratings
pivot = svd.pivot(index = 'user_id', columns ='book_id', values = 'rating').fillna(0)
pivot.head()

book_id,1,2,3,4,5,6,7,8,9,10,...,9981,9984,9985,9988,9990,9991,9992,9994,9995,10000
user_id,,,,,,,,,,,,,,,,,,,,,
35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173,4.0,5.0,4.0,4.0,0.0,5.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274,0.0,0.0,0.0,5.0,5.0,0.0,5.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
314,5.0,0.0,3.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
368,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


I then converted the pivot table to a matrix. I calculated each user's average rating and then replaced their rating int he matrix with the result of subtracting their average rating from the rating that they gave each book. 

In [18]:
# Convert the pivot table to a matrix
R = pivot.as_matrix()

# Identify a user's average rating 
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

/Users/ameenakhan/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


I am now ready to pass the values through the svds model.

In [19]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

sigma = np.diag(sigma)

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot.columns)

Now that I have the predicted ratings for each book, I can start making recommendations. I started by building a function that identifies the row for the desired user, sorts their predicted ratings, removes the books they've already ready, and then from the remaining predicted ratings returns the top values as recommendations.

In [20]:
def recommend_books(predictions_df, user_id, books_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = user_id - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the book information.
    user_data = original_ratings_df[original_ratings_df.user_id == (user_id)]
    user_full = (user_data.merge(books_df, how = 'left', left_on = 'book_id', right_on = 'book_id').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} books.'.format(user_id, user_full.shape[0]))
    print('Recommending the highest {0} predicted ratings books not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating books that the user hasn't seen yet.
    recommendations = (books_df[~books_df['book_id'].isin(user_full['book_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'book_id',
              right_on = 'book_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations



User 487 has already rated 136 books.
Recommending the highest 10 predicted ratings books not already rated.


I am going to test the function on User 487:

In [21]:
already_rated, recommendations = recommend_books(preds_df, 487, books, ratings, 10)

User 487 has already rated 136 books.
Recommending the highest 10 predicted ratings books not already rated.


In [22]:
# View books the user has already rated
already_rated.head(10)

,user_id,book_id,rating,title
0,487,119,5,The Handmaid's Tale
37,487,18,5,Harry Potter and the Prisoner of Azkaban (Harr...
55,487,67,5,A Thousand Splendid Suns
54,487,2419,5,Of Love and Other Demons
107,487,14,5,Animal Farm
52,487,5260,5,"The Pigman (The Pigman, #1)"
128,487,3435,5,"Fortunately, the Milk"
50,487,2128,5,"Plainsong (Plainsong, #1)"
49,487,4137,5,A Wolf at the Table
48,487,436,5,The God of Small Things


In [23]:
# View the 10 recommended books for User 487
recommendations

,book_id,title
34,63,Wuthering Heights
2,10,Pride and Prejudice
124,171,Emma
45,76,Sense and Sensibility
27,54,The Hitchhiker's Guide to the Galaxy (Hitchhik...
19,43,Jane Eyre
180,230,Persuasion
58,95,The Picture of Dorian Gray
99,142,The Pillars of the Earth (The Kingsbridge Seri...
130,177,Crime and Punishment


## Next Steps

My next steps will be to create a model that incorporates book features (year of publication, author, genre, etc) to see if I can get even more specific recommendations for each user. 